In [5]:
!pip install tk


In [ ]:
import random
from textblob import TextBlob
from movie_reviews import reviews
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext

def analyze_sentiment(reviews, movie_name, num_reviews):
    results = {'positive': 0, 'negative': 0, 'neutral': 0}
    selected_reviews = []

    movie_reviews = reviews.get(movie_name)
    
    if not movie_reviews:
        return None, 0, []
    
    random.shuffle(movie_reviews)
    selected_reviews = movie_reviews[:num_reviews]

    sentiment_reviews = []
    for review_data in selected_reviews:
        review = review_data[0]
        blob = TextBlob(review)
        polarity = blob.sentiment.polarity
        
        if polarity >= 0.05:
            sentiment = 'positive'
        elif polarity <= -0.05:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
        
        sentiment_reviews.append((review, sentiment))
        results[sentiment] += 1

    total_reviews = sum(results.values())
    if total_reviews == 0:
        return results, total_reviews, sentiment_reviews

    percentages = {sentiment: (count / total_reviews) * 100 for sentiment, count in results.items()}
    
    return percentages, total_reviews, sentiment_reviews

# Create main application window
root = tk.Tk()
root.title("Movie Review Sentiment Analyzer")
root.geometry("800x600")

# Create main container
main_frame = ttk.Frame(root, padding="20")
main_frame.pack(fill=tk.BOTH, expand=True)

# Movie selection
ttk.Label(main_frame, text="Select Movie:").grid(row=0, column=0, sticky=tk.W)
movie_names = sorted(reviews.keys())
movie_var = tk.StringVar()
movie_dropdown = ttk.Combobox(main_frame, textvariable=movie_var, values=movie_names, state="readonly")
movie_dropdown.grid(row=0, column=1, sticky=tk.EW, padx=5, pady=5)
movie_dropdown.current(0)

# Number of reviews input
ttk.Label(main_frame, text="Number of Reviews:").grid(row=1, column=0, sticky=tk.W)
num_reviews_var = tk.StringVar()
num_reviews_entry = ttk.Entry(main_frame, textvariable=num_reviews_var)
num_reviews_entry.grid(row=1, column=1, sticky=tk.EW, padx=5, pady=5)

# Analysis button
def analyze_reviews():
    movie_name = movie_var.get()
    try:
        num_reviews = int(num_reviews_var.get())
        if num_reviews <= 0:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid positive integer")
        return

    percentages, total, sentiment_reviews = analyze_sentiment(reviews, movie_name, num_reviews)
    
    if total == 0:
        messagebox.showerror("Error", f"No reviews found for {movie_name}")
        return

    # Update results labels
    results_text.config(state=tk.NORMAL)
    results_text.delete(1.0, tk.END)
    results_text.insert(tk.END, f"Analysis Results for {movie_name} ({total} reviews analyzed):\n\n")
    results_text.insert(tk.END, f"Positive: {percentages['positive']:.2f}%\n")
    results_text.insert(tk.END, f"Negative: {percentages['negative']:.2f}%\n")
    results_text.insert(tk.END, f"Neutral: {percentages['neutral']:.2f}%\n")
    results_text.config(state=tk.DISABLED)

    # Update review display
    reviews_text.config(state=tk.NORMAL)
    reviews_text.delete(1.0, tk.END)
    for i, (review, sentiment) in enumerate(sentiment_reviews, 1):
        reviews_text.insert(tk.END, f"Review {i} ({sentiment.capitalize()}):\n{review}\n\n")
    reviews_text.config(state=tk.DISABLED)

analyze_btn = ttk.Button(main_frame, text="Analyze Reviews", command=analyze_reviews)
analyze_btn.grid(row=2, column=0, columnspan=2, pady=10)

# Results display
results_text = scrolledtext.ScrolledText(main_frame, height=8, wrap=tk.WORD)
results_text.grid(row=3, column=0, columnspan=2, sticky=tk.EW, pady=5)
results_text.config(state=tk.DISABLED)

# Reviews display
ttk.Label(main_frame, text="Individual Review Analysis:").grid(row=4, column=0, sticky=tk.W, pady=5)
reviews_text = scrolledtext.ScrolledText(main_frame, height=15, wrap=tk.WORD)
reviews_text.grid(row=5, column=0, columnspan=2, sticky=tk.NSEW)
reviews_text.config(state=tk.DISABLED)

# Configure grid weights
main_frame.columnconfigure(1, weight=1)
main_frame.rowconfigure(5, weight=1)

root.mainloop()